In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers import Lambda
from keras import backend as K
from keras import regularizers

from keras.optimizers.legacy import SGD

In [ ]:
def VggModified(weight_decay, x_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_shape, kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dropout(0.4))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [ ]:
import torch.nn as nn
from keras.callbacks import EarlyStopping

In [ ]:
vgg_m = VggModified(0.0007, [32, 32, 3], 10)

In [ ]:
from keras.optimizers import Adam

In [ ]:
class cifar10vgg:
    def __init__(self, model, lr, train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32, 32, 3]

        self.model = model
        if train:
            self.model = self.train(self.model, lr)
        else:
            self.model.load_weights('cifar10vgg.h5')

    def normalize(self, X_train, X_test):
        mean = np.mean(X_train, axis=(0, 1, 2, 3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train - mean) / (std + 1e-7)
        X_test = (X_test - mean) / (std + 1e-7)
        return X_train, X_test

    def normalize_production(self, x):
        mean = 120.707
        std = 64.15
        return (x - mean) / (std + 1e-7)

    def predict(self, x, normalize=True, batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x, batch_size)

    def train(self, model, lr):
        batch_size = 128
        maxepoches = 250
        learning_rate = lr
        lr_decay = 1e-6
        lr_drop = 20

        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True
        )
        datagen.fit(x_train)

        sgd = SGD(learning_rate=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

        historytemp = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                                steps_per_epoch=x_train.shape[0] // batch_size,
                                epochs=maxepoches,
                                validation_data=(x_test, y_test),
                                callbacks=[reduce_lr, early_stopping],
                                verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [ ]:
if __name__ == '__main__':


    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = cifar10vgg(vgg_m, 0.1)

    predicted_x = model.predict(x_test)
    residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)

    loss = sum(residuals)/len(residuals)
    print("the validation 0/1 loss is: ",loss)

Epoch 1/250
390/390 - 36s - loss: 6.5284 - accuracy: 0.3113 - val_loss: 4.7010 - val_accuracy: 0.1735 - lr: 0.1000 - 36s/epoch - 91ms/step
Epoch 2/250
390/390 - 32s - loss: 3.0746 - accuracy: 0.4821 - val_loss: 2.5065 - val_accuracy: 0.4909 - lr: 0.1000 - 32s/epoch - 81ms/step
Epoch 3/250
390/390 - 32s - loss: 1.9180 - accuracy: 0.5943 - val_loss: 1.8105 - val_accuracy: 0.5842 - lr: 0.1000 - 32s/epoch - 83ms/step
Epoch 4/250
390/390 - 33s - loss: 1.5172 - accuracy: 0.6717 - val_loss: 1.6329 - val_accuracy: 0.6365 - lr: 0.1000 - 33s/epoch - 84ms/step
Epoch 5/250
390/390 - 33s - loss: 1.4183 - accuracy: 0.7022 - val_loss: 1.3515 - val_accuracy: 0.7291 - lr: 0.1000 - 33s/epoch - 84ms/step
Epoch 6/250
390/390 - 33s - loss: 1.3926 - accuracy: 0.7187 - val_loss: 1.4133 - val_accuracy: 0.7204 - lr: 0.1000 - 33s/epoch - 86ms/step
Epoch 7/250
390/390 - 32s - loss: 1.3926 - accuracy: 0.7337 - val_loss: 1.3513 - val_accuracy: 0.7523 - lr: 0.1000 - 32s/epoch - 81ms/step
Epoch 8/250
390/390 - 33s -

In [ ]:
# Test accuracy calculation without using evaluate
correct_predictions = np.sum(np.argmax(predicted_x, axis=1) == np.argmax(y_test, axis=1))
test_accuracy = correct_predictions / len(y_test)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.9371


In [ ]:
# To get the number of parameters
total_params = vgg_m.count_params()
print(f'Total parameters: {total_params}')


Total parameters: 5749322
